In [1]:
from torchvision.transforms import v2
from torch.utils.data import Subset, random_split, DataLoader
from sklearn.model_selection import train_test_split
from src.datasets.imageio import get_X_paths, get_y_paths
from src.datasets.dataset import CloudCoverDataset
from src.datasets.transforms.minmax_normalize import MinMaxNormalize
from src.datasets.exploration.visualization import show_rgb_infrared_ground_truth, show_channels, show_stats, show_class_distribution


C:\Users\ultav\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
C:\Users\ultav\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedba

# Data Loading

Each x can have up to 4 different feature types, one for R, G, B, Infrared.  
Each feature is an image.  

In [2]:
train_X_paths = get_X_paths()
train_X_paths[:3]

[[WindowsPath('../data/final/public/train_features/adwp/B02.tif'),
  WindowsPath('../data/final/public/train_features/adwp/B03.tif'),
  WindowsPath('../data/final/public/train_features/adwp/B04.tif'),
  WindowsPath('../data/final/public/train_features/adwp/B08.tif')],
 [WindowsPath('../data/final/public/train_features/adwu/B02.tif'),
  WindowsPath('../data/final/public/train_features/adwu/B03.tif'),
  WindowsPath('../data/final/public/train_features/adwu/B04.tif'),
  WindowsPath('../data/final/public/train_features/adwu/B08.tif')],
 [WindowsPath('../data/final/public/train_features/adwz/B02.tif'),
  WindowsPath('../data/final/public/train_features/adwz/B03.tif'),
  WindowsPath('../data/final/public/train_features/adwz/B04.tif'),
  WindowsPath('../data/final/public/train_features/adwz/B08.tif')]]

In [3]:
train_y_paths = get_y_paths()
train_y_paths[:3]

[WindowsPath('../data/final/public/train_labels/adwp.tif'),
 WindowsPath('../data/final/public/train_labels/adwu.tif'),
 WindowsPath('../data/final/public/train_labels/adwz.tif')]

## Create Dataset

In [4]:
train_ds = CloudCoverDataset(X_paths=train_X_paths, y_paths=train_y_paths)
train_ds = Subset(train_ds, list(range(500)))

total_size = len(train_ds)
val_size = int(total_size * 0.2)  # let's say 20% for validation
train_size = total_size - val_size
train_subset, val_subset = random_split(train_ds, [train_size, val_size])

In [5]:
sample_size_for_analysis = 8

In [6]:
train_dl = DataLoader(train_subset, batch_size=sample_size_for_analysis, shuffle=True, num_workers=11, persistent_workers=True)
val_dl = DataLoader(val_subset, batch_size=sample_size_for_analysis, shuffle=False, num_workers=11, persistent_workers=True)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import torch.optim as optim
import torchmetrics

# MODÈLE UNET MAISON. LE UNET SE DIVISE EN TROIS PARTIES (DÉBUT, DOWN, UP).

# Premières couches du modèle
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


# DESCENTE
class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


# MONTÉE
class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)

        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


# LA SORTIE
class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)


# TOUTES LES COUCHES COMBINÉES DANS L'ORDRE
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024 // 2)
        self.up1 = Up(1024, 512 // 2, bilinear)
        self.up2 = Up(512, 256 // 2, bilinear)
        self.up3 = Up(256, 128 // 2, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

C:\Users\ultav\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [8]:
class LightningUNet(pl.LightningModule):
    def __init__(self, n_channels, n_classes, train_loader, val_loader=None, test_loader=None, bilinear=True, learning_rate=1e-3):
        super(LightningUNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.bilinear = bilinear
        self.learning_rate = learning_rate
        self.jaccard_index = torchmetrics.JaccardIndex(num_classes=n_classes, task='binary')

        # Architecture Unet
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024 // 2)
        self.up1 = Up(1024, 512 // 2, bilinear)
        self.up2 = Up(512, 256 // 2, bilinear)
        self.up3 = Up(256, 128 // 2, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

    def training_step(self, batch, batch_idx):
        inputs, target = batch
        target_for_loss = target.long()
        output = self(inputs)
        
        # Jaccard Index Computation
        predicted_labels = torch.argmax(output, dim=1)
        self.jaccard_index(predicted_labels, target)
        self.log('train_jaccard', self.jaccard_index, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
        loss = nn.CrossEntropyLoss()(output, target_for_loss)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, target = batch
        target_for_loss = target.long()
        output = self(inputs)

        loss = F.cross_entropy(output, target_for_loss)
        
        # Jaccard Index Computation
        predicted_labels = torch.argmax(output, dim=1)
        self.jaccard_index(predicted_labels, target)
        
        # Log metrics
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_jaccard', self.jaccard_index, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        inputs, target = batch
        target_for_loss = target.long()
        output = self(inputs)
        
        loss = F.cross_entropy(output, target)

        # Jaccard Index Computation
        predicted_labels = torch.argmax(output, dim=1)
        self.jaccard_index(predicted_labels, target_for_loss)

        # Log metrics
        self.log('test_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('test_jaccard', self.jaccard_index, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def train_dataloader(self):
        return self.train_loader

    def val_dataloader(self):
        return self.val_loader if self.val_loader else None

    def test_dataloader(self):
        return self.test_loader if self.test_loader else None

In [9]:
from pytorch_lightning import Trainer
model = LightningUNet(4, 2, train_dl, val_dl, bilinear=True, learning_rate=1e-3)
trainer = Trainer(max_epochs=10)
trainer.fit(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\ultav\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name          | Type               | Params
------------------------------------------------------
0  | jaccard_index | BinaryJaccardIndex | 0     
1  | inc           | DoubleConv         | 39.6 K
2  | down1         | Down               | 221 

Sanity Checking: |                                                                               | 0/? [00:00<…

Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

C:\Users\ultav\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryJaccardIndex was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028


Validation: |                                                                                    | 0/? [00:00<…

C:\Users\ultav\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [10]:
model_path = "model/model.ckpt"
trainer.save_checkpoint(model_path)